In [ ]:
#coding:utf-8


from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import posts
from wordpress_xmlrpc.methods import taxonomies
from wordpress_xmlrpc import WordPressTerm
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc.methods import media, posts


wp = Client('http://localhost/wordpress-magazine/xmlrpc.php', 'admin', 'admin')

filename = './s28173928.jpg' #上传的图片文件路径
# prepare metadata
data = {
        'name': 's28173928.jpg',
        'type': 'image/jpeg',  # mimetype
}
# read the binary file and let the XMLRPC library encode it into base64
with open(filename, 'rb') as img:
        data['bits'] = xmlrpc_client.Binary(img.read())
response = wp.call(media.UploadFile(data))
# response == {
#       'id': 6,
#       'file': 'picture.jpg'
#       'url': 'http://www.example.com/wp-content/uploads/2012/04/16/picture.jpg',
#       'type': 'image/jpeg',
# }

attachment_id = response['id']
post = WordPressPost()
post.title = '1942—1945：我的上海沦陷生活'
post.content = '1942—1945：我的上海沦陷生活，颜滨 著、采金 整理，人民出版社，59.00元'
post.post_status = 'publish'  #文章状态，不写默认是草稿，private表示私密的，draft表示草稿，publish表示发布
post.terms_names = {
    'post_tag': ['Novel', 'firstpost'], #文章所属标签，没有则自动创建
    'category': ['Books', 'Tests'] #文章所属分类，没有则自动创建
 }
post.thumbnail = attachment_id #缩略图的id
post.id = wp.call(posts.NewPost(post))